In [1]:
import glob
import gzip
import itertools
import csv
import io
import pickle
import hashlib
import pandas as pd
import numpy as np

# WMT16 system-level data

### Checksums

In [2]:
mine = '2acd4f1d8fcc07115cc06bcaed4ff236'
yours = hashlib.md5(open("data/downloads/wmt16-metrics-results.tar.gz", 'rb').read()).hexdigest()
print(mine + '\n' + yours)
print(mine == yours)

# unzip to data/
# os.system('tar -xvf data/downloads/wmt16-metrics-results.tar.gz -p data/')

2acd4f1d8fcc07115cc06bcaed4ff236
2acd4f1d8fcc07115cc06bcaed4ff236
True


### Aggregate system-level data

In [3]:
da_files = [ i for i in glob.glob('data/wmt16-metrics-results/sys-level-results/standard/results-official/newstest2016*') if not 'noDA' in i ]
lp_df = []

for file in da_files:
    lp = file[-8:-4]
    lp = lp[0:2] + '-' + lp[2:]
    
    df = pd.read_csv(file, delimiter=' ')
    df['lp'] = lp
    
    lp_df.append(df)
sys_scores_da = pd.concat(lp_df)
sys_scores_da

/home/nlg-05/jtwei/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,BEER,CharacTer,DA,MPEDA,MT,RR,TER,UoW-ReVal,chrF1,chrF2,...,lp,mosesBLEU,mosesCDER,mosesPER,mosesWER,mtevalBLEU,mtevalNIST,wordF1,wordF2,wordF3
0,0.585902,-0.549603,0.050822,0.339520,online-B,0.208,0.4844,0.552969,56.9150,56.3313,...,cs-en,0.2843,0.4950,0.6103,0.4454,0.2916,8.1473,33.6227,33.1336,32.9737
1,0.597640,-0.521433,0.207242,0.351097,uedin-nmt,0.618,0.4971,0.562763,57.9286,57.6022,...,cs-en,0.3160,0.5265,0.6332,0.4640,0.3297,8.2267,35.9153,36.0229,36.0590
2,0.577497,-0.564633,-0.024469,0.335366,PJATK,0.005,0.4625,0.533773,55.5868,55.1861,...,cs-en,0.2847,0.4913,0.6193,0.4263,0.2954,7.8778,33.1321,33.1470,33.1520
3,0.569510,-0.567391,0.000170,0.331734,online-A,-0.070,0.4409,0.524500,54.6632,54.7261,...,cs-en,0.2581,0.4710,0.5907,0.3962,0.2818,7.6764,30.9812,31.1236,31.1714
4,0.590179,-0.536049,0.101488,0.347264,jhu-pbmt,0.318,0.4815,0.551973,57.3880,57.2473,...,cs-en,0.3058,0.5124,0.6218,0.4434,0.3178,8.1369,35.0911,35.2642,35.3222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,0.645333,-0.469980,0.128884,0.380690,online-B,0.606,0.5751,0.607241,64.0680,63.2894,...,ro-en,0.3906,0.5859,0.6858,0.5437,0.3952,9.3990,43.3236,42.6049,42.3706
3,0.599548,-0.515640,-0.012314,0.355509,online-A,-0.195,0.4913,0.548824,58.0101,58.4165,...,ro-en,0.3090,0.5227,0.6337,0.4527,0.3291,8.1913,35.5742,35.7740,35.8411
4,0.599308,-0.515031,-0.159592,0.350124,jhu-pbmt,-0.333,0.5055,0.522796,57.9284,58.0925,...,ro-en,0.3230,0.5366,0.6420,0.4727,0.3342,8.2017,36.6962,36.8571,36.9110
5,0.607985,-0.507060,-0.000472,0.361565,uedin-syntax,-0.097,0.5137,0.556067,59.5306,59.5155,...,ro-en,0.3369,0.5439,0.6561,0.4776,0.3466,8.4785,37.8558,37.9567,37.9905


Rename columns:

In [4]:
replace = {'DA':'score', 'MT':'system', 'mtevalBLEU':'BLEU', 'mtevalNIST':'NIST', 'mosesCDER':'CDER', 'mosesPER':'PER', 'mosesWER':'WER'}
sys_scores_da.columns = [ replace[i] if i in replace else i for i in sys_scores_da.columns ]

### WMT16 system-level data (all/raw)

In [5]:
lp_df = []

for file in glob.glob('data/wmt-human-evaluation/da-human-judgments/ad-seg-scores-*.csv'):
    lp = file[-9:-4]
    print(lp)
    
    df = pd.read_csv(file, delimiter=' ')
    df['lp'] = [lp] * len(df)
    
    lp_df.append(df)
raw_seg_scores_da = pd.concat(lp_df)

de-en
fi-en
ru-en
ro-en
tr-en
en-ru
cs-en


In [6]:
N = raw_seg_scores_da.groupby(['lp', 'SYS'], as_index=False).count()['N']
raw_sys_scores_da = raw_seg_scores_da.groupby(['lp', 'SYS'], as_index=False).mean()
raw_sys_scores_da['N'] = N

raw_sys_scores_da.columns = ['lp', 'system', 'sid', 'raw_score', 'z_score', 'N', '5']
raw_sys_scores_da = raw_sys_scores_da[['lp', 'system', 'raw_score', 'z_score', 'N']]
raw_sys_scores_da

,lp,system,raw_score,z_score,N
0,cs-en,PJATK,68.954545,-0.024469,2464
1,cs-en,cu-mergedtrees,55.783751,-0.502992,2437
2,cs-en,jhu-pbmt,72.568873,0.101488,2432
3,cs-en,online-A,69.455488,0.000170,2460
4,cs-en,online-B,70.844961,0.050822,2451
...,...,...,...,...,...
58,tr-en,jhu-syntax,40.803632,-0.363599,1762
59,tr-en,online-A,52.190503,0.002009,1790
60,tr-en,online-B,57.128603,0.163079,1804
61,tr-en,online-G,54.964850,0.108645,1835


In [7]:
sys_scores_da = sys_scores_da.merge(raw_sys_scores_da[['lp','system','raw_score','z_score']])
sys_scores_da[['lp','raw_score', 'z_score']].groupby('lp').corr('pearson')

raw_score   z_score
lp                                  
cs-en raw_score   1.000000  0.999818
      z_score     0.999818  1.000000
de-en raw_score   1.000000  0.999354
      z_score     0.999354  1.000000
en-ru raw_score   1.000000  0.993851
      z_score     0.993851  1.000000
fi-en raw_score   1.000000  0.999598
      z_score     0.999598  1.000000
ro-en raw_score   1.000000  0.998583
      z_score     0.998583  1.000000
ru-en raw_score   1.000000  0.996735
      z_score     0.996735  1.000000
tr-en raw_score   1.000000  0.999315
      z_score     0.999315  1.000000

### Validate correlations

In [8]:
sys_scores_da.groupby('lp') \
    ['lp'] \
    .count()

lp
cs-en     6
de-en    10
en-ru    12
fi-en     9
ro-en     7
ru-en    10
tr-en     8
Name: lp, dtype: int64

In [9]:
sys_scores_da[sys_scores_da.lp.str.endswith('en')] \
    .groupby('lp') \
    .corr()[2::len(sys_scores_da.columns)-2] \
    .round(3) \
    .T \
    .sort_index()

lp,cs-en,de-en,fi-en,ro-en,ru-en,tr-en
,score,score,score,score,score,score
BEER,0.990,0.879,0.972,0.852,0.901,0.982
BLEU,0.989,0.808,0.864,0.840,0.837,0.895
CDER,0.988,0.827,0.860,0.800,0.855,0.826
CharacTer,0.995,0.929,0.927,0.883,0.930,0.827
MPEDA,0.993,0.937,0.976,0.932,0.929,0.982
NIST,0.978,0.801,0.929,0.807,0.854,0.938
PER,0.970,0.730,0.767,0.748,0.887,0.940
RR,0.998,0.964,0.998,0.917,0.953,0.987
TER,0.969,0.834,0.846,0.793,0.847,0.788


In [10]:
sys_scores_da[~sys_scores_da.lp.str.endswith('en')] \
    .groupby('lp') \
    .corr()[2::len(sys_scores_da.columns)-2] \
    .round(3) \
    .T \
    .sort_index()

lp,en-ru
,score
BEER,0.922
BLEU,0.838
CDER,0.874
CharacTer,0.966
MPEDA,0.866
NIST,0.897
PER,0.854
RR,0.984
TER,0.879


### Pickle

In [11]:
pickle.dump(raw_seg_scores_da, open('data/pickles/wmt16-sys_level-all.pkl', 'wb'))
pickle.dump(sys_scores_da, open('data/pickles/wmt16-sys_level-agg.pkl', 'wb'))

# WMT16 segment-level data

In [12]:
da_files = [ i for i in glob.glob('data/wmt16-metrics-results/seg-level-results/da-results/metrics.*.csv') if not 'noDA' in i ]
lp_df = []

for file in da_files:
    lp = file[-9:-4]
    
    df = pd.read_csv(file, delimiter=' ')
    df['lp'] = lp
    lp_df.append(df)
    
    df['source'] = list(open('data/wmt16-metrics-results/seg-level-results/da-results/src.%s' % lp, 'rt'))
    df['output'] = list(open('data/wmt16-metrics-results/seg-level-results/da-results/snt.%s' % lp, 'rt'))
    df['reference'] = list(open('data/wmt16-metrics-results/seg-level-results/da-results/ref.%s' % lp, 'rt'))
    
seg_scores_da = pd.concat(lp_df)
seg_scores_da

/home/nlg-05/jtwei/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,BEER,DPMFcomb,DTED,HUMAN.Z,MPEDA,MT,SID,TYPE,UoW-ReVal,chrF1,...,lp,metrics-f,output,reference,sentBLEU,source,upf-cobalt,wordF1,wordF2,wordF3
0,0.345948,11.344566,0.3333,0.691142,0.269039,jhu-syntax,677,SYSTEM,0.408724,22.4808,...,de-en,1.022674,"He added that he hoped the company ""is a hit f...","He added that he wants the Corporation to ""fin...",0.158512,"Er fügte hinzu, er wünsche sich, dass das Unte...",0.332146,11.1111,11.9048,12.1951
1,0.592667,13.212386,0.3793,-0.762188,0.390229,jhu-pbmt,1092,SYSTEM,0.656208,54.9155,...,de-en,2.777495,On Wednesday morning there still 40 refugees w...,On Wednesday morning another 40 refugees were ...,0.421134,Am Mittwochmorgen hätten sich dort noch 40 Flü...,0.702998,47.0481,53.0944,55.4746
2,0.674845,13.643155,0.4091,-0.599309,0.392287,online-B,1375,SYSTEM,0.313238,63.9342,...,de-en,2.951276,"The Hearts Verteidigers led 2: 1, was sent fro...",Hearts were leading 2-1 when Paterson was dism...,0.505926,"Die Hearts führten 2:1, als Paterson vom Feld ...",0.671175,55.3685,52.3368,51.3989
3,0.632175,12.596852,0.4286,-0.305292,0.360400,online-G,616,SYSTEM,0.668733,59.6619,...,de-en,2.327898,And for every doll that looks like him and goe...,And for every doll that looks like him and pas...,0.178942,"Und bei jeder Puppe, die aussieht wie er und ü...",0.699800,24.2325,24.2325,24.2325
4,0.722258,14.745385,0.4545,0.362249,0.475718,jhu-syntax,679,SYSTEM,0.728891,68.5667,...,de-en,3.435297,"Gyorgy Bakondi, country security adviser for O...","Gyorgy Bakondi, homeland security adviser to O...",0.547704,"Gyorgy Bakondi, Landes-Sicherheitsberater für ...",0.762629,56.8175,55.7268,55.3725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,0.479873,12.053778,0.3784,0.221244,0.264372,uedin-nmt,1769,SYSTEM,0.513372,44.3992,...,cs-en,1.599906,To a larger model is heaviest used element bat...,"In the larger model, the largest component use...",0.153622,U většího modelu je pak nejtěžším použitým prv...,0.559142,17.9630,18.9047,19.2410
556,0.478653,11.930422,0.3793,-1.424952,0.350080,cu-mergedtrees,2099,SYSTEM,0.489572,43.0898,...,cs-en,2.127106,"oil is not a distinctive flavour, but a great ...","While it doesn't have a strong taste, hemp oil...",0.407322,"Konopný olej není nijak chuťově výrazný, ale s...",0.565177,39.2982,38.4291,38.1479
557,0.416998,10.087517,0.3273,-1.107383,0.193255,cu-mergedtrees,1330,SYSTEM,0.343713,33.0137,...,cs-en,-0.522042,But the feelings of frustration.\n,But frustrations were on the rise.\n,0.078825,Ale přichází pocity frustrace.\n,0.264592,10.0343,9.9256,9.8899
558,0.600188,12.600644,0.3793,0.111809,0.343030,jhu-pbmt,519,SYSTEM,0.497744,60.1689,...,cs-en,2.490539,He said that view in 2010 and now wants all te...,"He made that point in 2010, now he wants every...",0.407122,"Tento názor řekl v roce 2010 a nyní chce, aby ...",0.705017,39.2982,40.2079,40.5207


In [13]:
seg_scores_da.groupby('lp').count()[['SID']]

,SID
lp,
cs-en,560
de-en,560
en-ru,560
fi-en,560
ro-en,560
ru-en,560
tr-en,560


Rename columns:

In [14]:
replace = {'HUMAN.Z':'score', 'MT':'system'}
seg_scores_da.columns = [ replace[i] if i in replace else i for i in seg_scores_da.columns ]
seg_scores_da = seg_scores_da.drop(columns=['TYPE'], axis=1)
seg_scores_da

,BEER,DPMFcomb,DTED,score,MPEDA,system,SID,UoW-ReVal,chrF1,chrF2,...,lp,metrics-f,output,reference,sentBLEU,source,upf-cobalt,wordF1,wordF2,wordF3
0,0.345948,11.344566,0.3333,0.691142,0.269039,jhu-syntax,677,0.408724,22.4808,24.0387,...,de-en,1.022674,"He added that he hoped the company ""is a hit f...","He added that he wants the Corporation to ""fin...",0.158512,"Er fügte hinzu, er wünsche sich, dass das Unte...",0.332146,11.1111,11.9048,12.1951
1,0.592667,13.212386,0.3793,-0.762188,0.390229,jhu-pbmt,1092,0.656208,54.9155,62.9282,...,de-en,2.777495,On Wednesday morning there still 40 refugees w...,On Wednesday morning another 40 refugees were ...,0.421134,Am Mittwochmorgen hätten sich dort noch 40 Flü...,0.702998,47.0481,53.0944,55.4746
2,0.674845,13.643155,0.4091,-0.599309,0.392287,online-B,1375,0.313238,63.9342,61.4165,...,de-en,2.951276,"The Hearts Verteidigers led 2: 1, was sent fro...",Hearts were leading 2-1 when Paterson was dism...,0.505926,"Die Hearts führten 2:1, als Paterson vom Feld ...",0.671175,55.3685,52.3368,51.3989
3,0.632175,12.596852,0.4286,-0.305292,0.360400,online-G,616,0.668733,59.6619,57.9331,...,de-en,2.327898,And for every doll that looks like him and goe...,And for every doll that looks like him and pas...,0.178942,"Und bei jeder Puppe, die aussieht wie er und ü...",0.699800,24.2325,24.2325,24.2325
4,0.722258,14.745385,0.4545,0.362249,0.475718,jhu-syntax,679,0.728891,68.5667,65.1342,...,de-en,3.435297,"Gyorgy Bakondi, country security adviser for O...","Gyorgy Bakondi, homeland security adviser to O...",0.547704,"Gyorgy Bakondi, Landes-Sicherheitsberater für ...",0.762629,56.8175,55.7268,55.3725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,0.479873,12.053778,0.3784,0.221244,0.264372,uedin-nmt,1769,0.513372,44.3992,47.8013,...,cs-en,1.599906,To a larger model is heaviest used element bat...,"In the larger model, the largest component use...",0.153622,U většího modelu je pak nejtěžším použitým prv...,0.559142,17.9630,18.9047,19.2410
556,0.478653,11.930422,0.3793,-1.424952,0.350080,cu-mergedtrees,2099,0.489572,43.0898,38.3830,...,cs-en,2.127106,"oil is not a distinctive flavour, but a great ...","While it doesn't have a strong taste, hemp oil...",0.407322,"Konopný olej není nijak chuťově výrazný, ale s...",0.565177,39.2982,38.4291,38.1479
557,0.416998,10.087517,0.3273,-1.107383,0.193255,cu-mergedtrees,1330,0.343713,33.0137,33.1859,...,cs-en,-0.522042,But the feelings of frustration.\n,But frustrations were on the rise.\n,0.078825,Ale přichází pocity frustrace.\n,0.264592,10.0343,9.9256,9.8899
558,0.600188,12.600644,0.3793,0.111809,0.343030,jhu-pbmt,519,0.497744,60.1689,62.2127,...,cs-en,2.490539,He said that view in 2010 and now wants all te...,"He made that point in 2010, now he wants every...",0.407122,"Tento názor řekl v roce 2010 a nyní chce, aby ...",0.705017,39.2982,40.2079,40.5207


### Pickle

In [15]:
pickle.dump(seg_scores_da, open('data/pickles/wmt16-seg_level-agg.pkl', 'wb'))